# Mathematicon

**Примечание:** На данный момент код не содержит взаимодействия с математической разметкой и представляет собой самую первичную и самую примитивную реализацию поисковика непосредственно по plain texts.

In [27]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import numpy as np
import os
import pandas as pd
from pymorphy2 import MorphAnalyzer
import random
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import time

Изначально работали с текстами, очищенными от метаинформации.

**Примечание:** впоследствии тексты будут разделены на предложения с помощью `nltk.tokenize.sent_tokenize`, и индексироваться будут именно предложения, а не целиковые тексты; помимо того, будет создана база данных из двух таблиц:

- В первой будут содержаться id текста, название видео, ссылка на видео, раздел математики и уровень сложности материала;

- Во вторую таблицу будут записаны id предложения, id текста, текст предложения.

In [29]:
path = r"C:\Users\AnAnG\OneDrive\Рабочий стол\texts_to_count"

files = sorted(os.listdir(path))
#print(len(files))
#print(*files, sep="\n")

texts = []

for file in files:
    with open(os.path.join(path, file), "r", encoding="UTF-8") as f:
        text = re.search(r'text: "(.|\n)*"', f.read()).group(0).replace('text: "', "").replace('"', "")
        texts.append(text)

#print(len(texts))
#print(*texts, sep="\n")

In [33]:
morph = MorphAnalyzer()
stop_words = set(stopwords.words("russian"))

Препроцессинг текстов включал в себя перевод в нижний регистр, очистку от небуквенных и нечисловых элементов и лемматизацию буквенных элементов (слов) с помощью анализатора `MorphAnalyzer()` бибилиотеки `pymorphy2`.

In [35]:
def preprocessing(text):
    """
    Функция для получения лемм текста и очистки его от пунктуации
    Аргумент - строка с текстом описания
    Вывод - строка с очищенным готовым к обработке текстом
    """
    text = text.lower()
    tokenizer = RegexpTokenizer(r"[а-я0-9ё]+")
    text_tokenized = tokenizer.tokenize(text)
    lemmas = [
        morph.parse(word)[0].normal_form
        for word in text_tokenized
        if word.isalpha() == True
    ]
    return " ".join(lemmas)

In [37]:
preprocessed_texts = []

for i in range(len(texts)):
    preprocessed_texts.append(preprocessing(texts[i]))

Тексты мы индексировали посредством составления их tf-idf матрицы и сохранения ее в отдельный файл, чтобы не пришлось при каждом новом запуске кода считать матрицу заново.

In [39]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preprocessed_texts)

# получаем из TF-IDF матрицы массив
tfidf_array = tfidf_matrix.toarray()

# в файл сохраняем только массив
np.save("tfidf_matrix.npy", tfidf_array)

В функции поиска близость оценивалась на основании косинусного расстояния. На данный момент максимум в поисковой выдаче - это 5 текстов.

In [41]:
def search_tfidf(query, vectorizer, tfidf_matrix, texts):
    """
    Функция для поиска по TF-IDF
    Аргументы - строка с текстом запроса, векторизатор для текста запроса,
    матрица препроцесснутых текстов, тексты для выдачи
    Вывод - результаты поиска на основе косинусного расстояния
    """
    # считаем матрицу запроса
    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_idx = similarities.argsort()[-5:][::-1] # выбор топ-5 наиболее подходящих запросов
    results = [texts[i] for i in top_idx]
    return results

Повторюсь, реализация пока максимально примитивная, не включает работу с разметкой математическими тегами, а с точки зрения выдачи, пока не содержит метаинформацию текстов и показывает тексты целиком, а не предложения. В ячейке ниже приведен пример выдачи на запрос *пополам*.

In [45]:
# примитивная первичная реализация поисковика

query_text = input("Введите запрос: ")

# поиск по TF-IDF
results_tfidf = search_tfidf(
    query_text, vectorizer, np.load("tfidf_matrix.npy"), texts
)

print(*results_tfidf, sep="\n\n-------\n\n")

Введите запрос:  пополам


В этом видео мы посмотрим решение неравенств методом интервалов, видео преимущественно для учеников девятых классов, которые в конце года сдают экзамен. Итак, квадратное неравенство какое-нибудь, например, такое, ну и знак меньше либо равно нуля. Такое задание встречается в первой части в блоке “алгебра” под номером восемь. Итак, квадратное неравенство решается с помощью уравнения, то есть мы всё это дело приравниваем к нулю. Вычисляем дискриминант, то есть бэ квадрат минус четыре а цэ, получается девять минус четыре умножить на минус четыре, всё в итоге равно двадцать пять. Теперь находим корни. Формула корней – минус бэ плюс минус корень из дискриминанта поделить на два а. Первый корень – это, берём второй коэффициент с обратным знаком, то есть, если здесь стоит минус три, то с обратным знаком будет, наоборот, три. Итак, три плюс пять поделить на два, и второй корень – это три минус пять поделить на два. Здесь получится восемь пополам – это четыре, здесь минус два пополам, то есть ми